### MLBook-1
#### Ex 5
用上题实现的自助法，对Kaggle数字识别问题选择“训练集”和“测试集/验证集”。在本地根据训练集训练模型M，在本地对“测试集/验证集”计算正确率。用M对test.csv预测结果，上传到kaggle，查看kaggle分数。

** 备注：仅做演示，对训练集文件做了删减。 **

In [9]:
import pandas as pd
import numpy as np


def bootstrap(length):
    """
    Return:
        indexes of train/target datasets.
    """
    train = set()
    for m in range(length):
        train.add(np.random.randint(0, length))
    test = set(range(length)) - train
    return list(train), list(test)


dr_train_df = pd.read_csv("dr_train.csv")
idx_train, idx_test = bootstrap(dr_train_df.shape[0])
dr_train_x = dr_train_df.values[idx_train, 1:]
dr_train_y = dr_train_df.values[idx_train, 0]
dr_test_x = dr_train_df.values[idx_test, 1:]
dr_test_y = dr_train_df.values[idx_test, 0]

dr_pred_df = pd.read_csv("dr_test.csv")
dr_pred_x = dr_pred_df.values

In [10]:
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics


def timeit(func):
    import time
    def wrapper(*args, **kwargs):
        t1 = time.time()
        ret = func(*args, **kwargs)
        t2 = time.time()
        print(">>> Cost %.3f secs" % (t2 - t1))
        return ret
    return wrapper

@timeit
def score_accuracy(model, tr_x, tr_y, test_x, test_y):
    model.fit(tr_x, tr_y)
    y_pred = model.predict(test_x)
    y_true = test_y
    accuracy = metrics.accuracy_score(y_true, y_pred)
    print(">>> Accuracy: %.4f" % accuracy)

models = {
    "SDG": linear_model.SGDClassifier(),
    "SVC": svm.SVC(),
    "LogisticRegression": linear_model.LogisticRegression(),
    "SVC(gamma=0.001, C=100)": svm.SVC(gamma=0.001, C=100)
}

for name, m in models.items():
    print(">>> " + name)
    score_accuracy(m, dr_train_x, dr_train_y, dr_test_x, dr_test_y)
    print()

>>> SDG
>>> Accuracy: 0.8304
>>> Cost 0.115 secs

>>> SVC
>>> Accuracy: 0.1119
>>> Cost 6.229 secs

>>> LogisticRegression
>>> Accuracy: 0.8304
>>> Cost 1.140 secs

>>> SVC(gamma=0.001, C=100)
>>> Accuracy: 0.1119
>>> Cost 6.265 secs



In [11]:
lr = linear_model.LogisticRegression()
lr.fit(dr_train_x, dr_train_y)
dr_pred_y = lr.predict(dr_pred_x)

df = pd.DataFrame({"ImageId": range(1, len(dr_pred_y) + 1), "Label": dr_pred_y})
df.to_csv("dr_submission1.csv", index = False)